<a href="https://colab.research.google.com/github/Harika-Mullaguri/Deep-Learning/blob/main/GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import gradio as gr
import numpy as np
import tensorflow as tf

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Embedding
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import gradio as gr

In [3]:
text = """
Once upon a time in a land far away, there was a little village surrounded by lush green forests.
The villagers lived in harmony with nature, cherishing the beauty around them.
"""

In [4]:
# Step 1: Preprocessing the text--NLP
tokenizer = Tokenizer(char_level=True)  # Tokenize at the character level
tokenizer.fit_on_texts(text)
char_index = tokenizer.word_index
index_char = tokenizer.index_word
vocab_size = len(char_index) + 1

In [5]:
sequences = tokenizer.texts_to_sequences(text)

In [6]:
sequences = [item for sublist in sequences for item in sublist]

In [7]:
seq_length = 40
X, y = [], []
for i in range(len(sequences) - seq_length):
    X.append(sequences[i:i + seq_length])
    y.append(sequences[i + seq_length])

X = np.array(X)
y = to_categorical(y, num_classes=vocab_size)


In [8]:
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=50, input_length=seq_length),
    GRU(150, return_sequences=False),
    Dense(vocab_size, activation='softmax')
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [9]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [10]:
model.fit(X, y, epochs=1000, batch_size=64)

Epoch 1/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 61ms/step - accuracy: 0.0917 - loss: 3.2556
Epoch 2/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.1846 - loss: 3.2351
Epoch 3/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.1486 - loss: 3.2121
Epoch 4/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.1525 - loss: 3.1797
Epoch 5/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.1643 - loss: 3.1218
Epoch 6/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.1428 - loss: 3.0276
Epoch 7/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.1545 - loss: 2.9196
Epoch 8/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.1564 - loss: 2.9179
Epoch 9/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.1428 - loss: 2.9202
Epoch 10/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.1447 - loss: 2.8977
Epoch 11/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.1545 - loss: 2.8526
Epoch 12/1000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy:

In [11]:
def generate_text(seed_text, num_chars):
    for _ in range(num_chars):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=seq_length, truncating='pre')
        predicted_probs = model.predict(token_list, verbose=0)
        predicted_char_index = np.argmax(predicted_probs)
        predicted_char = index_char[predicted_char_index]
        seed_text += predicted_char
    return seed_text

In [12]:
def gradio_generate(seed_text, num_chars):
    return generate_text(seed_text, int(num_chars))

interface = gr.Interface(
    fn=gradio_generate,
    inputs=[
        gr.Textbox(label="Seed Text", placeholder="Enter seed text, e.g., 'Once upon a time'"),
        gr.Slider(minimum=50, maximum=500, step=10, value=200, label="Number of Characters")
    ],
    outputs=gr.Textbox(label="Generated Text"),
    title="GRU-Based Text Generator",
    description="Enter a seed text and specify the number of characters to generate new text using a GRU-based neural network."
)

# Launch the Gradio app
interface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://86eb79df70caaa71d2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
